In [2]:
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
import joblib

In [3]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Data generator to preprocess input images
datagen = ImageDataGenerator(rescale=1.0 / 255)

train_gen = datagen.flow_from_directory(
    r'S:\Datasets\Datasets\Animal Plant Dataset\New Plant Diseases Dataset(Augmented)\Test',
    target_size=(128, 128),  # Full resolution
    batch_size=128,           # Optimal batch size for GPU
    class_mode='categorical',
    shuffle=False
)



# Extract features using ResNet50
print("Extracting features using ResNet50...")
train_features = base_model.predict(train_gen, verbose=1)
train_labels = train_gen.classes



Found 17572 images belonging to 38 classes.
Extracting features using ResNet50...
138/138 [==============================] - 485s 4s/step


In [4]:
# Flatten features for PCA and SVM
train_features_flattened = train_features.reshape(train_features.shape[0], -1)

# Apply PCA to reduce dimensionality
print("Reducing dimensionality with PCA...")
pca = PCA(n_components=512)  # Reduced components to avoid memory issues
train_features_reduced = pca.fit_transform(train_features_flattened)

# Train SVM (on CPU)
print("Training SVM...")
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', probability=True))
svm_model.fit(train_features_reduced, train_labels)
print("SVM training complete!")


Reducing dimensionality with PCA...
Training SVM...
SVM training complete!


In [5]:
# Save PCA model
joblib.dump(pca, 'pca_model.pkl')
print("PCA model saved as 'pca_model.pkl'")

# Save SVM model
joblib.dump(svm_model, 'svm_model.pkl')
print("SVM model saved as 'svm_model.pkl'")


PCA model saved as 'pca_model.pkl'
SVM model saved as 'svm_model.pkl'


In [6]:
# Save class indices
import json
with open('class_indices.json', 'w') as f:
    json.dump(train_gen.class_indices, f)
